In [1]:
from model import juridIQ
from preprocessing import lemmatization
import scraping

import os
import re
import numpy as np
import pandas as pd

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/bertram/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/bertram/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/bertram/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /home/bertram/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!
[nltk_data] Downloading package words to /home/bertram/nltk_data...
[nltk_data]   Package words is already up-to-date!


In [2]:
files = pd.read_csv("data/KPMG Tax Case - Data Set.csv")

files.dropna(inplace=True)
files.drop_duplicates(inplace=True)

files["Numac"] = files.Numac.astype("int64").astype("object")

files.head()

,Date,Title,Numac,Link FR,Link NL
0,1/14/2020,REGION DE BRUXELLES-CAPITALE\r\nREGION DE BRUX...,2020010053,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
1,1/16/2020,SERVICE PUBLIC FEDERAL INTERIEUR\r\n10 DECEMBR...,2020010044,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
2,1/16/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n7 DECEMBRE ...,2020040052,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
3,1/24/2020,MINISTERE DE LA COMMUNAUTE FRANCAISE\r\n20 DEC...,2020010214,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...
4,1/28/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n20 JANVIER ...,2020040138,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...


In [3]:
files.shape

(770, 5)

In [4]:
saved_numacs = []

for file in os.listdir("data"):
    if file.endswith(".txt"):
        saved_numacs.append(file.split(".")[0])

files = files[files["Numac"].astype("string").isin(saved_numacs)]
files.shape

(769, 5)

In [5]:
texts = []

for numac in files.Numac:
    with open(f"Translation/translations_txt/{numac}.txt") as article:
        texts.append(article.read())
        
len(texts)

769

In [6]:
texts = lemmatization.lemmatizer(texts[20:], batchsize=5)

In [7]:
juridiq = juridIQ.JuridIQ("new")

topics = juridiq.get_topics(texts)
len(topics)

Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-06-16 11:33:15,172 - BERTopic - Transformed documents to Embeddings
2022-06-16 11:33:24,013 - BERTopic - Reduced dimensionality
2022-06-16 11:33:24,138 - BERTopic - Clustered reduced embeddings


Batches:   0%|          | 0/24 [00:00<?, ?it/s]

2022-06-16 11:34:42,247 - BERTopic - Reduced dimensionality
2022-06-16 11:34:42,441 - BERTopic - Calculated probabilities with HDBSCAN
2022-06-16 11:34:42,442 - BERTopic - Predicted clusters


749

In [9]:
sep = ", "

topics_as_string = []

for topic in topics:
    temp = [str(prob) for prob in topic]
    topics_as_string.append(f"[{sep.join(temp)}]")

#topics_as_string = [re.sub(' +', ', ', str(topic)).replace(".,", ",").replace(".]", "]") for topic in topics]

files = files.iloc[20:]

files["Topics"] = topics_as_string

In [10]:
import json

topics_as_list = [json.loads(topic) for topic in files["Topics"]]

In [11]:
files.to_csv("KPMG_with_topics.csv")

In [12]:
files_from_csv = pd.read_csv("KPMG_with_topics.csv").drop("Unnamed: 0", axis=1)
#files_from_csv.Topics.astype("Sparse")

In [13]:
files_from_csv.head()

,Date,Title,Numac,Link FR,Link NL,Topics
0,3/6/2020,SERVICE PUBLIC FEDERAL INTERIEUR\r\n11 FEVRIER...,2020040437,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,"[0.9999998949906659, 2.1300252244975015e-09, 3..."
1,3/6/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n17 MARS 201...,2020020270,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,"[0.24620416751460983, 0.005365887287729386, 0...."
2,3/6/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n23 MARS 201...,2020020271,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,"[0.9999999979317818, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
3,3/10/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n11 FEVRIER ...,2020020495,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,"[0.07527632215927113, 0.006072397949579861, 0...."
4,3/12/2020,SERVICE PUBLIC FEDERAL FINANCES\r\n21 FEVRIER ...,2020020502,http://www.ejustice.just.fgov.be/cgi/article.p...,http://www.ejustice.just.fgov.be/cgi/article.p...,"[0.07358741920367345, 0.006046967206458651, 0...."


In [16]:
import json

topics_as_list = [json.loads(topic) for topic in files_from_csv["Topics"]]

indexes = juridIQ.get_similar(topics_as_list[0], topics_as_list)

links_nl = []
links_fr = []

for index in indexes:
    links_nl.append(files_from_csv["Link NL"][index])
    
    links_fr.append(files_from_csv["Link FR"][index])
    
print(len(links_nl))

60


In [17]:
lemmatization.lemmatizer([translated_text])

NameError: name 'translated_text' is not defined

In [19]:
juridiq.save_model()